In [267]:
nomenclatura_asistencia = {
    2:	'X',
    17:	'A',
    1:	'1TRAB',
    6:	'1DFES',
    5:	'1VAC',
    7:	'1FINJ',
    3:	'B',
    9:	'1INC',
    4:	'C',
    8:	'1FJUS',
    14:	'CS',
    13:	'PGS AP',
    12:	'PGS SA',
    11:	'PSGS',
    16:	'PF'
}

trab = {
    '1TRAB': 1,
    'X': 1,
    'RT': 1, 
    'PGS SA': 1,
    '1DFES': 1
}

In [268]:
import pandas as pd 
import numpy as np 
from datetime import datetime, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ACTUALIZACION DEL PATH DIARIO

In [269]:
#Actualizacion de path diario
asistencia = pd.read_csv(r"C:\Users\centro.control\Downloads\IncidenciasPC-01-10-2025-al-27-10-2025.xls", sep='\t') #asistencia diaria
altas_people= pd.read_excel(r"C:\Users\centro.control\Downloads\altas-people-center1761665576.xlsx") #altas diarias
bajas_people = pd.read_excel(r"C:\Users\centro.control\Downloads\bajas-indicadores-people-center1761665579.xlsx") #bajas diarias
ub = pd.read_excel(r"C:\Users\centro.control\Downloads\activos-universida-bennetts28-10-2025_9_36am.xlsx",  converters={"usuario": lambda x: str(x)}) #universidad bennetts diaria

# ACUMULADOS

In [270]:
id_sitios = pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Sitios SAP.xlsx") # id sitios
altas = pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\02 ASISTENCIAS\Altas.xlsx") #altas acumuladas
bajas = pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\02 ASISTENCIAS\Bajas.xlsx") #bajas acumuladas
ub_mes  = pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\08 UNIVERSIDAD BENNETTS\10 Octubre 2025.xlsx") #Universidad bennetts acumulada

In [271]:
id_sitios = id_sitios[['SAP','NOMBRE COMERCIAL','id','SITIO','tipo', 'EJECUTIVO OPS 2025']] 

# ACTUALIZACIÓN DE CADA SECCION

## ALTAS

In [272]:
personas = altas_people[~altas_people['Nombre completo'].isin(altas['nombre_completo'])]

In [273]:
personas= personas[['Sitio',
                'Nombre completo',
                'Turno',
                'fecha de llegada al sitio'
                ]]
personas[['candidate_id', 'empleo','no_semana', 'incidencia', 'asistencia']] = np.nan
personas['Nomenclatura'] = 'A'
personas[['validacion', 'asistencia']] = 1

In [274]:
personas = pd.merge(personas, id_sitios, left_on='Sitio', right_on='NOMBRE COMERCIAL', how='left')

In [275]:
personas= personas.rename(
    columns={'SITIO': 'sitio',
            'Nombre completo': 'nombre_completo',
            'Turno': 'turno',
            'fecha de llegada al sitio': 'fecha',
            'Nomenclatura': 'nomenclatura',
            'Sitio': 'sitio sap',
            'EJECUTIVO OPS 2025': 'ejecutivo'
    }
)
personas = personas[['id', 'candidate_id', 'sitio', 'nombre_completo', 'empleo', 'turno',
    'no_semana', 'incidencia', 'fecha', 'nomenclatura','sitio sap','SAP','tipo', 'validacion', 'asistencia', 'ejecutivo']]

In [276]:
altas = pd.concat([altas,personas])

## ASISTENCIA

In [277]:
# renombra la columna del id por candidate_id, para evitar problemas al unir con los sitios y no confundir id de la persona con id del sitio 
asistencia = asistencia.rename(columns={
        'id':'candidate_id'
})

In [278]:
#cruzamos la tablausando el nombre comercial como id
asistencia= asistencia.merge(id_sitios, left_on='sitio', right_on='NOMBRE COMERCIAL', how='left').drop(columns=['NOMBRE COMERCIAL'])

asistencia = asistencia[~ asistencia['incidencia'].isin([17,3])] #quitamos las {bajas: 17, altas:3 }

In [279]:
# Funciones para mapear nomenclatura y asistencia
# cambia la incidencia por la nomenclatura correspondiente, es decir, si tiene una incidencia numerica, la cambia por una nomenclatura del diccionario nomenclatura_asistencia
def nomenclatura(incidencia):
    return nomenclatura_asistencia.get(incidencia,None)
asistencia['nomenclatura'] = list(map(nomenclatura,asistencia['incidencia']))
# cambia la nomenclatura por 1 o 0 dependiendo si la nomenclatura cuenta o no para la asistencia. 
def work(job):
    return trab.get(job,0)
asistencia['asistencia'] = list(map(work,asistencia['nomenclatura']))
asistencia['validacion'] = asistencia['asistencia']


In [280]:
#renombrar columnas 
asistencia = asistencia.rename(
    columns={
        'sitio': 'sitio sap',
        'EJECUTIVO OPS 2025': 'ejecutivo',
        'SITIO': 'sitio'
    }
)

In [281]:
# Filtrar y reordenar columnas
asistencia= asistencia[[
    'id','candidate_id','sitio', 'nombre_completo','empleo','turno',
    'no_semana','incidencia','fecha','nomenclatura','sitio sap',
    'SAP','tipo','validacion',	'asistencia','ejecutivo']]

In [282]:
asistencia = asistencia.drop_duplicates()


## BAJAS

In [283]:
#Eliminar los nombres duplicados de las bajas de people center 
bajas_people = bajas_people.drop_duplicates(subset=['Nombre'], keep = 'last')

In [284]:
#Buscamos a las personas que aun no han sido dadas de baja en el acumulado de bajas, pero que se presentan como nuevas bajas en el documento que se descargaron de people center
personas_bajas = bajas_people[~bajas_people['Nombre'].isin(bajas['nombre_completo'])].copy()

In [285]:
#rellenar columnnas faltantes para que tengan el mismo formato que la asistencia y altas
personas_bajas= personas_bajas[['Sitio',
                'Nombre',
                'Turno',
                'ULTIMO DIA TRABAJADO'
                ]]
personas_bajas[['candidate_id', 'empleo', 'no_semana', 'incidencia']] = np.nan
personas_bajas['Nomenclatura'] = 'B'
personas_bajas[['validacion','asistencia']] = 1

In [286]:
#cruzamos las tablas para buscar el id del sitio, el nombre del sitio, ejecutivo y tipo
personas_bajas = pd.merge(personas_bajas, id_sitios, left_on='Sitio', right_on='NOMBRE COMERCIAL', how='left')


In [287]:
#simplemente renombramos las columnas para que coincidan con el formato final
personas_bajas= personas_bajas.rename(
    columns={'SITIO': 'sitio',
            'Nombre': 'nombre_completo',
            'Turno': 'turno',
            'ULTIMO DIA TRABAJADO': 'fecha',
            'Nomenclatura': 'nomenclatura',
            'NOMBRE COMERCIAL': 'sitio sap',
            'EJECUTIVO OPS 2025': 'ejecutivo'
            }
)
# Reordenamos las columnas para tener el mismo formato que las otras tablas
personas_bajas= personas_bajas[['id', 'candidate_id', 'sitio','nombre_completo',
            'empleo', 'turno','no_semana', 'incidencia', 'fecha',
            'nomenclatura', 'sitio sap', 'SAP','tipo', 'validacion', 'asistencia', 'ejecutivo']]

In [288]:
bajas = pd.concat([bajas, personas_bajas], ignore_index=True)

## UNIVERSIDAD BENNETTS

In [289]:
ayer = (datetime.today()- timedelta(days=1)).strftime('%d/%m/%Y')
ub['fecha llegada al sitio'] = pd.to_datetime(ub['fecha llegada al sitio'], errors='coerce')
ub['Fecha del reporte'] = ayer

In [290]:
ub['Más de 15 días'] = np.where(ub['fecha llegada al sitio']< (datetime.today() - timedelta(days=15)), 1, 0)
ub['Más de 30 días'] = np.where(ub['fecha llegada al sitio']< (datetime.today() - timedelta(days=30)), 1, 0)

In [291]:
ub = pd.merge(ub, id_sitios, left_on='sitio', right_on='NOMBRE COMERCIAL', how='left').drop(columns=['NOMBRE COMERCIAL','sitio']).rename(columns={'SITIO': 'sitio', 'Unnamed: 14': 'Certificado','fecha c. supervisor/operador':'fecha c. supervisor/operados'})

In [292]:
ub.rename(columns={'EJECUTIVO OPS 2025': 'ejecutivo'}, inplace=True)

In [293]:
ub=ub[[
    'id','Fecha del reporte','Folio de alta','nombre_completo', 'sitio', 'turno', 'empleo',
    'usuario','contraseña','fecha llegada al sitio','c supervisor/operador','fecha c. supervisor/operados',
    'c. seguridad y higiene', 'fecha c. seguridad y higiene', 'c. key quimica', 'fecha c. key quimica', 'Certificado',
    'Más de 15 días', 'Más de 30 días','ejecutivo'
]]

In [294]:
ub_mes = pd.concat([ub, ub_mes], ignore_index=True).drop_duplicates(keep='last')

## CEDULAS DE SERVICIO

In [295]:
field  = pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\01 CEDULAS DE FIELD\10 de Octubre 2025.xlsx")
field_nuevos  = pd.read_excel(r"C:\Users\centro.control\Downloads\CUMPLIMIENTO DE RUTINAS - 2025-10-28T093738.879.xlsx", header=[1])

In [296]:
field_nuevos['rutinas realizadas reales'] = field_nuevos['Rutinas Realizadas']
field_nuevos[' % de cumplimiento real'] =  field_nuevos['rutinas realizadas reales']/field_nuevos['Rutinas Asignadas']

In [297]:
field_nuevos = pd.merge(field_nuevos, id_sitios[['SITIO','EJECUTIVO OPS 2025']], left_on='Sitio', right_on= 'SITIO', how='left').drop(columns='SITIO').rename(columns={'EJECUTIVO OPS 2025': 'Ejecutivo', 'ID Sitio People Center': 'id'})

In [298]:
field_nuevos = field_nuevos[['ID Cedula', 'id', 'Sitio', 'Turno', 'Día', 'Fecha Entrada',
    'Hora Entrada', 'Fecha Salida', 'Hora Salida', 'Objetivo Rutinas',
    'Rutinas Asignadas', 'Rutinas Realizadas', 'Geolocalización',
    'Ejecutivo', 'rutinas realizadas reales', ' % de cumplimiento real']]

In [299]:
field = pd.concat([field,field_nuevos])

In [300]:
field['Fecha Entrada'] = field['Fecha Entrada'].dt.date
field['Fecha Salida'] = field['Fecha Salida'].dt.date

In [301]:
field = field.drop_duplicates(subset=['Sitio', 'Turno', 'Fecha Entrada'], keep='last', ignore_index=True)
field =  field[field['Sitio'] != 'PRUEBAS TECH2B']

# CTS

In [302]:
credential_files = r"C:\Users\centro.control\Documents\Python\Proyectos\Actualizacion tablero\Web scrapping\starry-strand-473423-n5-8f446379f536.json"
file_name = r"Copia de Tickets de Recursos Humanos (CTS) (respuestas)"
sheet_name = r"Tickets CTS RH"

In [303]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

In [304]:
credential = ServiceAccountCredentials.from_json_keyfile_name(credential_files, scope)
gc = gspread.authorize(credential)

In [305]:
sh = gc.open(file_name)
worksheet= sh.worksheet(sheet_name)
data = worksheet.get_all_values()
cts = pd.DataFrame(data[1:], columns=data[0])
cts = pd.merge(cts, id_sitios[['SITIO','id']], left_on='Sitio', right_on='SITIO',how="left")
cts.drop(columns='SITIO', inplace= True)
cols = ['id'] + [col for col in cts.columns if col !="id"]
cts= cts[cols]
cts = cts.rename(columns={
    'Puesto.1':'Puesto',
    'Sexo requerido.1':'Sexo requerido' ,
    'Horario de trabajo.1':'Horario de trabajo' ,
    'Rango de edad.1' : 'Rango de edad'
})

In [306]:
cts['FOLIO'] = pd.to_numeric(cts['FOLIO'], errors= "coerce")

<!-- # CTS -->


# GUARDAR DATOS

In [ ]:
# altas.to_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\02 ASISTENCIAS\Altas.xlsx", sheet_name='Hoja1', index=False)
# asistencia.to_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\02 ASISTENCIAS\10 Octubre 2025.xlsx", sheet_name="Hoja1", index=False)
# bajas.to_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\02 ASISTENCIAS\Bajas.xlsx", sheet_name="Hoja1", index=False)
# ub_mes.to_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\08 UNIVERSIDAD BENNETTS\10 Octubre 2025.xlsx", index=False, sheet_name='Hoja1')
# field.to_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\01 CEDULAS DE FIELD\10 de Octubre 2025.xlsx", index=False, sheet_name="Hoja1")
# cts.to_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\04 TICKETS RH\Tickets de Recursos Humanos (CTS) (respuestas).xlsx", sheet_name= 'Tickets CTS RH', index=False)